# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications. 

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below. 

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [3]:
# Imports here
import numpy as np
import pandas as pd
import matplotlib as pyplot

import time

import torch
from torchvision import datasets, transforms, models
import torch.nn.functional as F
from torch import nn 
from torch import optim
from torch.autograd import Variable

from collections import OrderedDict

from PIL import Image

## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz). The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.
 

In [4]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [5]:
# TODO: Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.Grayscale(3), transforms.RandomHorizontalFlip(), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
test_transforms = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
valid_transforms = transforms.Compose([transforms.Resize(256), transforms.CenterCrop(224), transforms.ToTensor(), transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# TODO: Load the datasets with ImageFolder
train_datasets = datasets.ImageFolder(train_dir, transform=train_transforms)
test_datasets = datasets.ImageFolder(test_dir, transform=test_transforms)
valid_datasets = datasets.ImageFolder(valid_dir, transform=valid_transforms)

# TODO: Using the image datasets and the trainforms, define the dataloaders
trainloader = torch.utils.data.DataLoader(train_datasets, batch_size=32, shuffle=True)
testloader = torch.utils.data.DataLoader(test_datasets, batch_size=32, shuffle=True)
validloader = torch.utils.data.DataLoader(valid_datasets, batch_size=32, shuffle=True)

### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [6]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. If you want to talk through it with someone, chat with your fellow students! You can also ask questions on the forums or join the instructors in office hours.

Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

In [ ]:
# IGNORE - define classifier network
class Network(nn.Module):
    def __init__(self, input_size, output_size, hidden_layers, drop_p=0.5):
        super().__init__()
        
        self.hidden_layers = nn.ModuleList([nn.Linear(input_size, hidden_layers[0])])
        
        layer_sizes = zip(hidden_layers[:-1], hidden_layers[1:])
        self.hidden_layers.extend([nn.Linear(h1, h2) for h1, h2 in layer_sizes])
        
        self.output = nn.Linear(hidden_layers[-1], output_size)
        
        self.dropout = nn.Dropout(p=drop_p)
        
    def forward(self, x):
        for each in self.hidden_layers:
            x = F.relu(each(x))
            x = self.dropout(x)
        logits = self.output(x)
        
        output = F.log_softmax(logits, dim=1)
        
        return output # revise to softmax if needed

In [ ]:
# IGNORE - create network, define criterion and optimizer
input_size = 784 #TBD
hidden_layers = [500, 200, 100] #TBD
learnrate = 0.1 #change as needed

model = Network(input_size=input_size, output_size=102, hidden_layers=hidden_layers, drop_p=0.5)
criterion = nn.NLLLoss() #change as needed
optimizer = optim.Adam(model.parameters(), lr=learnrate) #change as needed

In [7]:
# load pretrained model
net = models.densenet121(pretrained=True)
net

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to /root/.torch/models/densenet121-a639ec97.pth
100%|██████████| 32342954/32342954 [00:01<00:00, 21952291.23it/s]


DenseNet(
  (features): Sequential(
    (conv0): Conv2d (3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1))
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm.1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (relu.1): ReLU(inplace)
        (conv.1): Conv2d (64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm.2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
        (relu.2): ReLU(inplace)
        (conv.2): Conv2d (128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm.1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
        (relu.1): ReLU(inplace)
        (conv.1): Conv2d (96, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
     

In [8]:
# freeze features in pretrained model (no backprop)
for param in net.parameters():
    param.requires_grad = False
    
'''classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(25088, 4096)), ('relu', nn.ReLU(inplace=True)),
                                        ('dropout', nn.Dropout(p=0.5)), ('fc2', nn.Linear(4096, 4096)), 
                                        ('relu1', nn.ReLU(inplace=True)),('dropout2', nn.Dropout(p=0.5)), 
                                        ('fc3', nn.Linear(4096, 102)), ('output', nn.LogSoftmax(dim=1))])) #change if needed'''

classifier = nn.Sequential(OrderedDict([('fc1', nn.Linear(1024, 102)), ('output', nn.LogSoftmax(dim=1))]))
net.classifier = classifier

In [9]:
learnrate = 0.01 #change as needed

criterion = nn.NLLLoss() #change as needed
optimizer = optim.Adam(net.classifier.parameters(), lr=learnrate) #change as needed

In [10]:
# train classifier
cuda = torch.cuda.is_available()
if cuda:
    net = net.cuda()
else:
    net = net.cpu()

epochs = 10
print_every = 40
steps = 0
 

for e in range(epochs):
    net.train()
    running_loss = 0
    
    for ii, (images, labels) in enumerate(trainloader):
        steps += 1
        
        inputs, labels = Variable(images), Variable(labels)
        optimizer.zero_grad()
        
        if cuda:
            inputs, labels = inputs.cuda(), labels.cuda()
        else:
            inputs, labels = inputs.cpu(), labels.cpu()

        outputs = net.forward(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.data[0]
        
        if steps % print_every == 0:
            net.eval()
            
            accuracy = 0
            test_loss = 0
            
            for ii, (images, labels) in enumerate(validloader):
                
                inputs = Variable(images, volatile=True)
                labels = Variable(labels, volatile=True)
                
                if cuda:
                    inputs, labels = inputs.cuda(), labels.cuda()
                else:
                    inputs, labels = inputs.cpu(), labels.cpu()
                
                output = net.forward(inputs)
                test_loss += criterion(output, labels).data[0]
                
                ps = torch.exp(output).data
                #ps = output.data
                
                equality = (labels.data == ps.max(1)[1])
                
                accuracy += equality.type_as(torch.FloatTensor()).mean()
                
            print("Epoch: {}/{}.. ".format(e+1, epochs),
              "Training Loss: {:.3f}.. ".format(running_loss/print_every),
              "Test Loss: {:.3f}.. ".format(test_loss/len(validloader)),
              "Test Accuracy: {:.3f}".format(accuracy/len(validloader)))
            
            running_loss = 0
            
            net.train()


Epoch: 1/10..  Training Loss: 5.574..  Test Loss: 8.242..  Test Accuracy: 0.042
Epoch: 1/10..  Training Loss: 1.936..  Test Loss: 29.228..  Test Accuracy: 0.014
Epoch: 1/10..  Training Loss: 1.234..  Test Loss: 40.546..  Test Accuracy: 0.019
Epoch: 1/10..  Training Loss: 1.222..  Test Loss: 42.313..  Test Accuracy: 0.019
Epoch: 1/10..  Training Loss: 1.150..  Test Loss: 48.999..  Test Accuracy: 0.008
Epoch: 2/10..  Training Loss: 0.588..  Test Loss: 53.273..  Test Accuracy: 0.016
Epoch: 2/10..  Training Loss: 0.592..  Test Loss: 57.439..  Test Accuracy: 0.005
Epoch: 2/10..  Training Loss: 0.671..  Test Loss: 56.795..  Test Accuracy: 0.011
Epoch: 2/10..  Training Loss: 0.778..  Test Loss: 56.594..  Test Accuracy: 0.005
Epoch: 2/10..  Training Loss: 0.740..  Test Loss: 59.823..  Test Accuracy: 0.011
Epoch: 3/10..  Training Loss: 0.314..  Test Loss: 73.000..  Test Accuracy: 0.011
Epoch: 3/10..  Training Loss: 0.393..  Test Loss: 66.862..  Test Accuracy: 0.008


KeyboardInterrupt: 

## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [ ]:
# TODO: Do validation on the test set
net.eval()

cuda = torch.cuda.is_available()
if cuda:
    net.cuda()
else:
    net.cpu()

accuracy = 0
test_loss = 0
for ii, (images, labels) in enumerate(testloader):
    #images = images.resize_(images.size()[0], 784)

    inputs = Variable(images, volatile=True)
    labels = Variable(labels, volatile=True)

    if cuda:
        inputs, labels = inputs.cuda(), labels.cuda()
    else:
        inputs, labels = inputs.cpu(), labels.cpu()

    output = net.forward(inputs)
    test_loss += criterion(output, labels).data[0]

    #ps = torch.exp(output).data
    ps = output.data

    equality = (labels.data == ps.max(1)[1])

    accuracy += equality.type_as(torch.FloatTensor()).mean()
    
    print ("Test Accuracy: {:.3f}".format(accuracy/len(testloader)))

## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:
# TODO: Save the checkpoint 
net.class_to_idx = image_datasets['train'].class_to_idx #fix variable name

#checkpoint = {'input_size': 224*224, 'output_size': 102, 'features': [each.out_features for each in net.features],'classifier': [each.out_features for each in net.classifier], 'state_dict': net.state_dict(), 'optimizer': optimizer.state_dict(), 'mapping': net.class_to_idx}
checkpoint = {'features': net.features, 'classifier': net.classifier, 'state_dict': net.state_dict(), 'optimizer': optimizer.state_dict(), 'epochs': epochs}

torch.save(checkpoint, 'checkpoint.pth')

## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
# TEST CELL
model = nn.Sequential()
model.features = net.features
model.classifier = net.classifier
model

In [ ]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
def load_checkpoint(filepath):
    checkpoint = torch.load(filepath)
    
    model = nn.Sequential()
    
    model.features = checkpoint['features']
    model.classifier = checkpoint['classifier']
    model.load_state_dict(checkpoint['state_dict'])
    
    return model

# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like 

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network. 

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training. 

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation. 

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [ ]:
# test process image
import numpy as np
from PIL import Image

data_dir = 'flowers'
image = data_dir + '/train' + '/10' + '/image_07086.jpg'

img = Image.open(image)
np_image = np.array(img)
means = [0.485, 0.456, 0.406]
stds = [0.229, 0.224, 0.225]
#((np_image[0][0]/256)-means)/stds
new = np_image.transpose(2,0,1)
new.shape
np_image.shape
#print('original matrix: {}'.format(np_image))
#print('transposed matrix: {}'.format(np_image.transpose()))
#np_image.size
#img.size
type(img)

In [10]:
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''
    
    # TODO: Process a PIL image for use in a PyTorch model
    
    size = image.size
    if size[0] > size[1]:
        short, long = size[1], size[0]
    else:
        short, long = size[0], size[1]
    # RESIZE    
    new_short = 256
    new_long = (256/size[0])*size[1]
    img = image.resize(new_short, new_long)
    # CENTERCROP
    area = ((new_short-224)/2, (new_long-224)/2, 224, 224)
    img = img.crop_(area)
    # NORMALISE COLOR CHANNELS
    np_image = np.array(img)
    
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]
    
    for w in np_image:
        for h in np_image[w]:
            np_image[w][h] = (np_image[w][h]/255 - means)/stds
    
    # TRANSPOSE IMAGE
    img_tp = np_image.transpose(2, 0, 1)
    
    return img_tp

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [6]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()
    
    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))
    
    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    
    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)
    
    ax.imshow(image)
    
    return ax

In [11]:
# FIX THIS
data_dir = 'flowers'
directory = data_dir + '/train' + '/10' + '/image_07086.jpg'
image = Image.open(directory)
img = process_image(image)
ax = imshow(img)

ValueError: unknown resampling filter

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [ ]:
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''
    
    # TODO: Implement the code to predict the class from an image file

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [ ]:
# TODO: Display an image along with the top 5 classes